In [1]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import lines

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import time
import copy
from collections import deque
from utils import memoize, PriorityQueue
#from search import Problem, Node


In [2]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

In [3]:
class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action)
        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

In [4]:

def board_init():
    board = []
    col_num = 4
    for i in range(col_num):
        line = []
        for j in range(col_num):
            line.append(5)
        board.append(line)
    board[0]=[0]*col_num
    board[1]=[0]*col_num
    board[2]=[0]*col_num
    board[3]=[0]*col_num
    
    board[0][0]=1
    return board

def board_goal():
    board = []
    col_num = 4
    for i in range(col_num):
        line = []
        for j in range(col_num):
            line.append(5)
        board.append(line)
    board[0]=[1]*col_num
    board[1]=[1]*col_num
    board[2]=[1]*col_num
    board[3]=[1]*col_num
    
    board[0][3]=0
    board[3][0]=0
    print(board)
    return board
    
def board_printer(board):
    print(board[0])
    print(board[1])
    print(board[2])
    print(board[3])
    

In [5]:
class labProblem(Problem):
    def __init__(self, initial, goal):
        Problem.__init__(self, initial, goal)
        self.line_size = 4
        self.col_size = 4
        
    def goal_test(self, state):
        return self.goal == state
    
    #state == board
    def actions(self, state):
        
        possible_states = []
        
        for line_pos in range(self.line_size):
            for col_pos in range(self.col_size):
                if state[line_pos][col_pos]!=5 and state[line_pos][col_pos]!=1 :
                    curr_item_states = self.generate_states(state, line_pos, col_pos)
                    possible_states.extend(curr_item_states)   
        
        
            
        return possible_states
    
    
    def generate_states(self, state, line_pos, col_pos):
        states = []
        #right
        if col_pos<2:
            if state[line_pos][col_pos+2]==1 and state[line_pos][col_pos+1]==0:
                new_state = self.move(state, line_pos, col_pos, 'right')
                states.append(new_state)
        #left
        if col_pos>1:
            if state[line_pos][col_pos-2]==1 and state[line_pos][col_pos-1]==0:
                new_state = self.move(state, line_pos, col_pos, 'left')
                states.append(new_state)
        #up
        if line_pos>1:
            if state[line_pos-2][col_pos]==1 and state[line_pos-1][col_pos]==0:
                new_state = self.move(state, line_pos, col_pos, 'up')
                states.append(new_state)
        
        #down
        if line_pos<2:
            if state[line_pos+2][col_pos]==1 and state[line_pos+1][col_pos]==0:
                new_state = self.move(state, line_pos, col_pos, 'down')
                states.append(new_state)
        
        return states
    
    
    def move(self, state, line_pos, col_pos, direction):
        
        new_state = copy.deepcopy(state)
        new_state[line_pos][col_pos]=1
        
        if direction =='right':
            new_state[line_pos][col_pos+1]=1
            new_state[line_pos][col_pos+2]=0
            
            return new_state
        
        if direction == 'left':
            new_state[line_pos][col_pos-1]=1
            new_state[line_pos][col_pos-2]=0
            
            return new_state
        
        if direction == 'up':
            new_state[line_pos-1][col_pos]=1
            new_state[line_pos-2][col_pos]=0
            
            return new_state
        
        if direction == 'down':
            new_state[line_pos+1][col_pos]=1
            new_state[line_pos+2][col_pos]=0
            
            return new_state
    
    
    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        return action
    
    def h(self, node):
        """h function is straight-line distance from a node's state to goal."""
        current_state = node.state
        distance = 0
        for i in range(self.line_size):
            for j in range(self.col_size):
                if self.goal[i][j]!=current_state[i][j]:
                    distance+=1
        return distance

        

# Breadth search

In [6]:
def tree_breadth_search_for_vis(problem):
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Don't worry about repeated paths to a state. [Figure 3.7]"""
    # we use these two variables at the time of visualisations
    iterations = 0
    #Adding first node to the queue
    frontier = deque([Node(problem.initial)])
    iterations += 1
    proc_nodes = []
    
    while frontier:
        #Popping first node of queue
        node = frontier.popleft()
        print('---------------------Iteration {}'.format(iterations))
        board_printer(node.state)
        # modify the currently searching node to red
        iterations += 1
        if problem.goal_test(node.state):
            # modify goal node to green after reaching the goal
            iterations += 1
            return(iterations,  node)
        
        next_nodes = node.expand(problem)
        proc_nodes.append(node)
        next_frontiers = []
        if len(frontier) == 0:
            frontier.extend(next_nodes)
        else:
            for node in next_nodes:
                if len(list(filter(lambda x:node.state == x.state, proc_nodes))):
                    rep = rep + 1
                    continue
                if len(list(filter(lambda x:node.state == x.state, frontier))):
                    continue
                next_frontiers.append(node)
                
        frontier.extend(next_frontiers)
        
        
    return None

def breadth_first_tree_search(problem):
    "Search the shallowest nodes in the search tree first."
    iterations, node = tree_breadth_search_for_vis(problem)
    return(iterations,node)





## Breadth search path

In [7]:
board_initial_state = board_init()
board_goal_state = board_goal()
boardProb = labProblem(board_initial_state, board_goal_state)

print('Board initial state\n')
board_printer(board_initial_state)
print('\n\n\nBoard goal state\n')
board_printer(board_goal_state)




import time
start = time.time()
result = tree_breadth_search_for_vis(boardProb)
print("Found solution on {} secs".format(str(time.time() - start)))
print(result)

[[1, 1, 1, 0], [1, 1, 1, 1], [1, 1, 1, 1], [0, 1, 1, 1]]
Board initial state

[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]



Board goal state

[1, 1, 1, 0]
[1, 1, 1, 1]
[1, 1, 1, 1]
[0, 1, 1, 1]
---------------------Iteration 1
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
---------------------Iteration 2
[0, 1, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
---------------------Iteration 3
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
---------------------Iteration 4
[0, 0, 1, 0]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
---------------------Iteration 5
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
---------------------Iteration 6
[0, 0, 0, 0]
[0, 1, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
---------------------Iteration 7
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 1, 0]
[0, 0, 0, 0]
---------------------Iteration 8
[1, 1, 0, 0]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
---------------------Iteration 9
[0, 0, 1, 0]
[0, 0, 1, 1]
[0, 1, 0, 0]
[0, 0, 0, 0]
---------------------Iteration 1

[1, 0, 0, 0]
[1, 0, 1, 0]
[1, 1, 0, 1]
[0, 1, 0, 0]
---------------------Iteration 290
[1, 0, 0, 0]
[1, 0, 1, 0]
[0, 0, 1, 1]
[0, 0, 1, 1]
---------------------Iteration 291
[1, 0, 1, 0]
[1, 0, 1, 0]
[0, 1, 0, 0]
[0, 1, 1, 0]
---------------------Iteration 292
[1, 0, 0, 0]
[1, 0, 1, 1]
[0, 1, 0, 1]
[0, 0, 1, 0]
---------------------Iteration 293
[1, 0, 0, 0]
[1, 0, 1, 0]
[0, 1, 0, 0]
[1, 1, 0, 1]
---------------------Iteration 294
[1, 0, 1, 0]
[1, 1, 1, 0]
[0, 0, 0, 0]
[1, 1, 0, 0]
---------------------Iteration 295
[1, 0, 0, 0]
[1, 0, 1, 1]
[1, 1, 0, 0]
[0, 0, 1, 0]
---------------------Iteration 296
[1, 0, 0, 0]
[1, 0, 1, 1]
[0, 0, 1, 0]
[1, 1, 0, 0]
---------------------Iteration 297
[1, 0, 0, 0]
[1, 1, 0, 0]
[1, 0, 1, 1]
[0, 0, 1, 0]
---------------------Iteration 298
[1, 0, 0, 0]
[1, 1, 0, 0]
[0, 0, 1, 0]
[1, 0, 1, 1]
---------------------Iteration 299
[1, 1, 0, 1]
[0, 1, 1, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
---------------------Iteration 300
[0, 0, 1, 0]
[0, 1, 1, 1]
[1, 0, 0, 1]
[0, 

[1, 0, 1, 1]
[0, 0, 1, 0]
[1, 0, 1, 1]
[1, 0, 0, 0]
---------------------Iteration 490
[1, 0, 1, 1]
[0, 0, 1, 0]
[1, 1, 0, 0]
[0, 1, 1, 0]
---------------------Iteration 491
[1, 0, 1, 1]
[0, 0, 0, 0]
[1, 1, 1, 0]
[1, 0, 1, 0]
---------------------Iteration 492
[0, 1, 1, 1]
[0, 0, 1, 0]
[1, 0, 1, 1]
[0, 1, 0, 0]
---------------------Iteration 493
[0, 1, 1, 1]
[0, 0, 0, 0]
[1, 1, 1, 0]
[0, 1, 1, 0]
---------------------Iteration 494
[0, 1, 1, 1]
[0, 0, 1, 0]
[1, 1, 0, 0]
[0, 0, 1, 1]
---------------------Iteration 495
[0, 1, 1, 1]
[0, 0, 1, 1]
[1, 0, 1, 0]
[0, 0, 1, 0]
---------------------Iteration 496
[1, 1, 1, 0]
[1, 0, 1, 0]
[0, 1, 0, 0]
[1, 1, 0, 0]
---------------------Iteration 497
[1, 1, 1, 0]
[1, 1, 1, 0]
[0, 0, 0, 0]
[0, 1, 1, 0]
---------------------Iteration 498
[1, 1, 1, 0]
[1, 1, 0, 0]
[0, 0, 1, 0]
[1, 0, 1, 0]
---------------------Iteration 499
[0, 1, 1, 0]
[1, 1, 0, 0]
[1, 1, 0, 0]
[1, 1, 0, 0]
---------------------Iteration 500
[0, 1, 1, 0]
[0, 0, 1, 0]
[1, 0, 1, 1]
[1, 

---------------------Iteration 689
[1, 1, 1, 0]
[1, 1, 0, 0]
[0, 1, 0, 1]
[0, 0, 1, 1]
---------------------Iteration 690
[1, 1, 1, 0]
[0, 0, 1, 1]
[0, 0, 1, 1]
[1, 1, 0, 0]
---------------------Iteration 691
[1, 1, 1, 0]
[1, 0, 1, 1]
[1, 1, 0, 0]
[0, 1, 0, 0]
---------------------Iteration 692
[1, 1, 1, 0]
[0, 0, 0, 1]
[0, 1, 1, 0]
[1, 1, 1, 0]
---------------------Iteration 693
[1, 1, 1, 0]
[0, 0, 1, 1]
[0, 1, 0, 0]
[1, 0, 1, 1]
---------------------Iteration 694
[1, 1, 1, 0]
[0, 0, 1, 0]
[0, 1, 0, 1]
[1, 1, 0, 1]
---------------------Iteration 695
[1, 1, 0, 0]
[1, 1, 1, 0]
[1, 1, 1, 0]
[0, 1, 0, 0]
---------------------Iteration 696
[1, 1, 0, 0]
[0, 1, 1, 0]
[0, 1, 1, 0]
[1, 0, 1, 1]
---------------------Iteration 697
[1, 1, 0, 0]
[1, 0, 0, 1]
[1, 1, 1, 0]
[0, 0, 1, 1]
---------------------Iteration 698
[1, 1, 0, 0]
[1, 0, 0, 0]
[1, 1, 1, 1]
[0, 1, 0, 1]
---------------------Iteration 699
[1, 1, 0, 1]
[0, 0, 0, 1]
[0, 1, 1, 0]
[1, 1, 0, 1]
---------------------Iteration 700
[1, 1, 0

---------------------Iteration 889
[1, 1, 0, 0]
[0, 1, 1, 1]
[1, 0, 0, 1]
[1, 0, 1, 0]
---------------------Iteration 890
[1, 1, 0, 0]
[0, 0, 1, 0]
[1, 1, 0, 0]
[1, 1, 1, 1]
---------------------Iteration 891
[1, 1, 0, 0]
[1, 1, 1, 1]
[1, 0, 0, 0]
[1, 1, 0, 0]
---------------------Iteration 892
[1, 0, 0, 0]
[1, 0, 1, 1]
[1, 0, 1, 1]
[1, 1, 0, 0]
---------------------Iteration 893
[1, 0, 0, 0]
[1, 0, 0, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
---------------------Iteration 894
[1, 0, 0, 0]
[1, 0, 1, 1]
[1, 1, 0, 0]
[1, 0, 1, 1]
---------------------Iteration 895
[1, 0, 0, 0]
[1, 0, 1, 0]
[1, 1, 0, 1]
[1, 1, 0, 1]
---------------------Iteration 896
[1, 0, 1, 0]
[1, 1, 1, 0]
[1, 0, 0, 1]
[1, 1, 0, 0]
---------------------Iteration 897
[1, 0, 0, 1]
[1, 1, 0, 1]
[1, 0, 1, 0]
[1, 1, 0, 0]
---------------------Iteration 898
[1, 0, 0, 0]
[1, 1, 0, 0]
[1, 0, 1, 1]
[1, 0, 1, 1]
---------------------Iteration 899
[1, 0, 0, 0]
[1, 1, 1, 0]
[1, 1, 1, 0]
[1, 0, 0, 1]
---------------------Iteration 900
[1, 0, 0

---------------------Iteration 1055
[1, 1, 0, 0]
[1, 1, 1, 0]
[0, 1, 1, 1]
[0, 0, 1, 1]
---------------------Iteration 1056
[0, 1, 0, 0]
[1, 1, 1, 1]
[1, 1, 1, 1]
[0, 1, 0, 0]
---------------------Iteration 1057
[1, 1, 0, 0]
[0, 1, 1, 1]
[0, 1, 1, 1]
[0, 0, 1, 1]
---------------------Iteration 1058
[1, 1, 0, 0]
[1, 0, 0, 1]
[0, 1, 1, 1]
[1, 1, 0, 1]
---------------------Iteration 1059
[1, 1, 0, 0]
[0, 1, 1, 1]
[1, 1, 1, 0]
[0, 0, 1, 1]
---------------------Iteration 1060
[1, 1, 0, 0]
[0, 1, 1, 0]
[1, 1, 1, 1]
[0, 1, 0, 1]
---------------------Iteration 1061
[1, 1, 0, 1]
[0, 1, 0, 1]
[1, 1, 0, 0]
[0, 1, 1, 1]
---------------------Iteration 1062
[1, 1, 0, 0]
[0, 0, 1, 1]
[1, 1, 0, 1]
[0, 1, 1, 1]
---------------------Iteration 1063
[1, 1, 0, 1]
[0, 1, 0, 1]
[1, 0, 1, 1]
[1, 0, 0, 1]
---------------------Iteration 1064
[1, 1, 0, 1]
[0, 1, 0, 1]
[1, 1, 0, 0]
[1, 1, 1, 0]
---------------------Iteration 1065
[1, 1, 0, 0]
[1, 1, 0, 1]
[1, 1, 0, 1]
[1, 0, 0, 1]
---------------------Iteration 1

[1, 0, 1, 0]
---------------------Iteration 1219
[0, 1, 1, 1]
[0, 0, 0, 0]
[1, 1, 1, 0]
[1, 1, 1, 1]
---------------------Iteration 1220
[0, 1, 1, 1]
[1, 0, 1, 1]
[0, 0, 1, 0]
[1, 1, 0, 1]
---------------------Iteration 1221
[0, 1, 1, 1]
[1, 1, 0, 1]
[1, 1, 0, 1]
[0, 0, 1, 0]
---------------------Iteration 1222
[0, 1, 1, 1]
[1, 1, 0, 0]
[1, 1, 0, 0]
[1, 1, 0, 1]
---------------------Iteration 1223
[0, 1, 1, 1]
[1, 1, 0, 1]
[0, 0, 1, 1]
[1, 1, 0, 0]
---------------------Iteration 1224
[0, 1, 1, 1]
[1, 0, 0, 1]
[0, 1, 1, 1]
[0, 1, 1, 0]
---------------------Iteration 1225
[0, 0, 1, 1]
[1, 0, 0, 1]
[0, 1, 1, 1]
[1, 0, 1, 1]
---------------------Iteration 1226
[0, 1, 1, 1]
[0, 0, 1, 1]
[1, 1, 0, 1]
[1, 1, 0, 0]
---------------------Iteration 1227
[1, 1, 1, 0]
[1, 0, 1, 0]
[1, 1, 0, 1]
[1, 1, 0, 0]
---------------------Iteration 1228
[1, 1, 1, 0]
[1, 0, 1, 0]
[0, 0, 1, 1]
[1, 0, 1, 1]
---------------------Iteration 1229
[1, 1, 1, 0]
[1, 0, 1, 1]
[0, 1, 0, 1]
[1, 0, 1, 0]
-------------------

[1, 1, 1, 0]
[0, 0, 1, 1]
---------------------Iteration 1389
[1, 0, 1, 1]
[0, 1, 1, 1]
[0, 1, 1, 1]
[1, 0, 1, 0]
---------------------Iteration 1390
[1, 0, 1, 1]
[1, 0, 0, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
---------------------Iteration 1391
[1, 1, 1, 1]
[0, 1, 1, 0]
[1, 1, 1, 1]
[1, 0, 0, 0]
---------------------Iteration 1392
[1, 1, 1, 1]
[1, 0, 0, 0]
[1, 1, 1, 1]
[0, 1, 1, 0]
---------------------Iteration 1393
[1, 1, 1, 1]
[1, 1, 0, 0]
[1, 0, 1, 1]
[1, 1, 0, 0]
---------------------Iteration 1394
[0, 1, 1, 0]
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 0, 0, 0]
---------------------Iteration 1395
[1, 1, 1, 0]
[1, 1, 0, 1]
[1, 1, 0, 1]
[1, 1, 0, 0]
---------------------Iteration 1396
[1, 1, 1, 0]
[0, 0, 0, 1]
[1, 1, 1, 1]
[1, 1, 1, 0]
---------------------Iteration 1397
[1, 1, 1, 0]
[0, 0, 1, 1]
[1, 1, 0, 1]
[1, 0, 1, 1]
---------------------Iteration 1398
[1, 1, 1, 0]
[1, 0, 1, 1]
[0, 1, 0, 1]
[1, 1, 0, 1]
---------------------Iteration 1399
[1, 1, 1, 0]
[1, 1, 0, 1]
[0, 0, 1, 1]
[1, 0, 1, 1]
------

---------------------Iteration 1672
[1, 0, 0, 0]
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
---------------------Iteration 1673
[1, 1, 1, 1]
[1, 1, 1, 1]
[0, 0, 0, 1]
[1, 1, 1, 1]
---------------------Iteration 1674
[0, 1, 1, 1]
[0, 1, 1, 1]
[0, 1, 1, 1]
[1, 1, 1, 1]
---------------------Iteration 1675
[0, 0, 0, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
---------------------Iteration 1676
[1, 0, 1, 1]
[1, 0, 1, 1]
[1, 0, 1, 1]
[1, 1, 1, 1]
---------------------Iteration 1677
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 0, 0, 0]
[1, 1, 1, 1]
---------------------Iteration 1678
[1, 1, 1, 1]
[0, 1, 1, 0]
[1, 1, 1, 1]
[1, 1, 1, 1]
---------------------Iteration 1679
[1, 1, 0, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 0, 1]
---------------------Iteration 1680
[0, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 0]
---------------------Iteration 1681
[0, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
[0, 1, 1, 1]
---------------------Iteration 1682
[0, 1, 1, 0]
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
---------------------Iteration 1

## Breadth search result

In [8]:
result_node = result[1]
path = result_node.path()
print('Path length = ', len(path))
for p in path:
    print('--------------------')
    board_printer(p.state)
    if (hasattr(p.state,"direction_used")):
        print(p.state.direction_used)
        print(p.state.from_pos)
        print(p.state.to_pos)

Path length =  14
--------------------
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
--------------------
[0, 1, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
--------------------
[0, 0, 1, 0]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
--------------------
[1, 1, 0, 0]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
--------------------
[1, 0, 1, 1]
[0, 1, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
--------------------
[1, 0, 1, 1]
[0, 0, 1, 1]
[0, 1, 0, 0]
[0, 0, 0, 0]
--------------------
[1, 1, 1, 1]
[0, 1, 1, 1]
[0, 0, 0, 0]
[0, 0, 0, 0]
--------------------
[1, 1, 1, 1]
[0, 0, 1, 1]
[0, 1, 0, 0]
[0, 1, 0, 0]
--------------------
[1, 1, 1, 1]
[0, 0, 1, 1]
[0, 1, 0, 0]
[0, 0, 1, 1]
--------------------
[1, 1, 1, 1]
[0, 0, 1, 1]
[0, 1, 0, 0]
[1, 1, 0, 1]
--------------------
[1, 1, 1, 1]
[1, 0, 1, 1]
[1, 1, 0, 0]
[0, 1, 0, 1]
--------------------
[1, 1, 1, 1]
[1, 0, 0, 1]
[1, 1, 1, 0]
[0, 1, 1, 1]
--------------------
[1, 1, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
[0, 1, 1, 1]
--------------------
[1, 1, 1, 0]

# Depth search 

In [9]:
def tree_depth_search_for_vis(problem):
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    Don't worry about repeated paths to a state. [Figure 3.7]"""
    
    # we use these two variables at the time of visualisations
    iterations = 0
    
    
    #Adding first node to the stack
    frontier = [Node(problem.initial)]
    proc_nodes = []
    
    while frontier:
        #Popping first node of stack
        node = frontier.pop()
        print('---------------------Iteration {}'.format(iterations))
        board_printer(node.state)
        # modify the currently searching node to red
        
        iterations += 1
        proc_nodes.append(node)
        
        if problem.goal_test(node.state):
            # modify goal node to green after reaching the goal
        
            
        
            return(iterations, node)
        
        next_nodes = node.expand(problem)
        proc_nodes.append(node)
        next_frontiers = []
        if len(frontier) == 0:
            frontier.extend(next_nodes)
        else:
            for node in next_nodes:
                if len(list(filter(lambda x:node.state == x.state, proc_nodes))):
                    continue
                if len(list(filter(lambda x:node.state == x.state, frontier))):
                    continue
                next_frontiers.append(node)
                
        frontier.extend(next_frontiers)
        
    return None

def depth_first_tree_search(problem):
    "Search the deepest nodes in the search tree first."
    iterations, all_node_colors, node = tree_depth_search_for_vis(problem)
    return(iterations, all_node_colors, node)




## Depth search path

In [10]:
board_initial_state = board_init()
board_goal_state = board_goal()
boardProb = labProblem(board_initial_state, board_goal_state)

print('Board initial state\n')
board_printer(board_initial_state)
print('\n\n\nBoard goal state\n')
board_printer(board_goal_state)




import time
start = time.time()
result = tree_depth_search_for_vis(boardProb)
print("Found solution on {} secs".format(str(time.time() - start)))
print(result)

[[1, 1, 1, 0], [1, 1, 1, 1], [1, 1, 1, 1], [0, 1, 1, 1]]
Board initial state

[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]



Board goal state

[1, 1, 1, 0]
[1, 1, 1, 1]
[1, 1, 1, 1]
[0, 1, 1, 1]
---------------------Iteration 0
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
---------------------Iteration 1
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
---------------------Iteration 2
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 1, 0]
[0, 0, 0, 0]
---------------------Iteration 3
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 1, 0]
[1, 0, 0, 0]
---------------------Iteration 4
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 1, 0]
[0, 1, 1, 0]
---------------------Iteration 5
[0, 0, 1, 0]
[0, 0, 1, 0]
[1, 1, 0, 0]
[0, 1, 1, 0]
---------------------Iteration 6
[0, 0, 1, 0]
[0, 0, 1, 0]
[1, 0, 1, 1]
[0, 1, 1, 0]
---------------------Iteration 7
[0, 0, 1, 0]
[0, 1, 1, 0]
[1, 1, 1, 1]
[0, 0, 1, 0]
---------------------Iteration 8
[0, 0, 1, 0]
[0, 1, 1, 0]
[1, 1, 1, 1]
[1, 1, 0, 0]
---------------------Iteration 9

# Deapth search result

In [11]:
result_node = result[1]
path = result_node.path()
print('Path length = ', len(path))
for p in path:
    print('--------------------')
    board_printer(p.state)
    if (hasattr(p.state,"direction_used")):
        print(p.state.direction_used)
        print(p.state.from_pos)
        print(p.state.to_pos)

Path length =  14
--------------------
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
--------------------
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
--------------------
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 1, 0]
[0, 0, 0, 0]
--------------------
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 1, 0]
[1, 0, 0, 0]
--------------------
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 1, 0]
[0, 1, 1, 0]
--------------------
[0, 0, 1, 0]
[0, 0, 1, 0]
[1, 1, 0, 0]
[0, 1, 1, 0]
--------------------
[0, 0, 1, 0]
[0, 0, 1, 0]
[1, 0, 1, 1]
[0, 1, 1, 0]
--------------------
[0, 0, 1, 0]
[1, 1, 0, 0]
[1, 0, 1, 1]
[0, 1, 1, 0]
--------------------
[0, 0, 1, 1]
[1, 1, 0, 1]
[1, 0, 1, 0]
[0, 1, 1, 0]
--------------------
[0, 0, 1, 1]
[1, 1, 0, 0]
[1, 0, 1, 1]
[0, 1, 1, 1]
--------------------
[0, 0, 1, 1]
[1, 0, 1, 1]
[1, 0, 1, 1]
[0, 1, 1, 1]
--------------------
[1, 1, 0, 1]
[1, 0, 1, 1]
[1, 0, 1, 1]
[0, 1, 1, 1]
--------------------
[1, 0, 0, 1]
[1, 1, 1, 1]
[1, 1, 1, 1]
[0, 1, 1, 1]
--------------------
[1, 1, 1, 0]

# Greedy search

In [12]:
def best_first_graph_search_for_vis(problem, f):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    
    # we use these two variables at the time of visualisations
    iterations = 0
    f = memoize(f, 'f')
    node = Node(problem.initial)
    iterations += 1
    
    if problem.goal_test(node.state):
        iterations += 1
        return(iterations,  node)
    
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    iterations += 1
    explored = set()
    
    while frontier:
        node = frontier.pop()
        print('---------------------Iteration {}'.format(iterations))
        board_printer(node.state)
        print("h = ", f(node))
        iterations += 1
        
        if problem.goal_test(node.state):
            return(iterations,  node)
        
        explored.add(tuple(map(tuple,node.state)))
        
        for child in node.expand(problem):
            temp = tuple(map(tuple,child.state))
            if temp not in explored and child not in frontier:
                frontier.append(child)
                
                
        
        
    return None

def greedy_best_first_search(problem, h=None):
    """Greedy Best-first graph search is an informative searching algorithm with f(n) = h(n).
    You need to specify the h function when you call best_first_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    iterations,  node = best_first_graph_search_for_vis(problem, lambda n: h(n))
    return(iterations,  node)



## Greedy search path

In [13]:
board_initial_state = board_init()
board_goal_state = board_goal()
boardProb = labProblem(board_initial_state, board_goal_state)

print('Board initial state\n')
board_printer(board_initial_state)
print('\n\n\nBoard goal state\n')
board_printer(board_goal_state)

import time
start = time.time()
result =greedy_best_first_search(boardProb)
print("Found solution on {} secs".format(str(time.time() - start)))
print(result)

[[1, 1, 1, 0], [1, 1, 1, 1], [1, 1, 1, 1], [0, 1, 1, 1]]
Board initial state

[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]



Board goal state

[1, 1, 1, 0]
[1, 1, 1, 1]
[1, 1, 1, 1]
[0, 1, 1, 1]
---------------------Iteration 2
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
h =  13
---------------------Iteration 3
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
h =  12
---------------------Iteration 4
[0, 0, 0, 0]
[0, 1, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
h =  11
---------------------Iteration 5
[0, 0, 0, 0]
[0, 0, 1, 0]
[1, 1, 0, 0]
[0, 1, 0, 0]
h =  10
---------------------Iteration 6
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 1, 0]
[0, 1, 1, 0]
h =  9
---------------------Iteration 7
[0, 0, 1, 0]
[0, 0, 1, 0]
[1, 1, 0, 0]
[0, 1, 1, 0]
h =  8
---------------------Iteration 8
[0, 0, 1, 0]
[0, 0, 1, 0]
[1, 0, 1, 1]
[0, 1, 1, 0]
h =  7
---------------------Iteration 9
[0, 0, 1, 0]
[0, 1, 1, 0]
[1, 1, 1, 1]
[0, 0, 1, 0]
h =  6
---------------------Iteration 10
[1, 0, 1, 0]
[1, 1, 1, 

# Greedy search result

In [14]:

result_node = result[1]
path = result_node.path()
print('Path length = ', len(path))
for p in path:
    print('--------------------')
    board_printer(p.state)
    if (hasattr(p.state,"direction_used")):
        print(p.state.direction_used)
        print(p.state.from_pos)
        print(p.state.to_pos)

Path length =  14
--------------------
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
--------------------
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
--------------------
[0, 0, 0, 0]
[0, 1, 1, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
--------------------
[0, 0, 0, 0]
[0, 0, 1, 0]
[1, 1, 0, 0]
[0, 1, 0, 0]
--------------------
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 1, 0]
[0, 1, 1, 0]
--------------------
[0, 0, 1, 0]
[0, 0, 1, 0]
[1, 1, 0, 0]
[0, 1, 1, 0]
--------------------
[0, 0, 1, 0]
[0, 0, 1, 0]
[1, 0, 1, 1]
[0, 1, 1, 0]
--------------------
[0, 0, 1, 0]
[1, 1, 0, 0]
[1, 0, 1, 1]
[0, 1, 1, 0]
--------------------
[1, 1, 0, 0]
[1, 1, 0, 0]
[1, 0, 1, 1]
[0, 1, 1, 0]
--------------------
[1, 1, 1, 0]
[1, 1, 1, 0]
[1, 0, 0, 1]
[0, 1, 1, 0]
--------------------
[1, 1, 1, 1]
[1, 1, 1, 1]
[1, 0, 0, 0]
[0, 1, 1, 0]
--------------------
[1, 1, 1, 1]
[1, 1, 1, 0]
[1, 0, 0, 1]
[0, 1, 1, 1]
--------------------
[1, 1, 1, 1]
[1, 1, 1, 0]
[1, 1, 1, 0]
[0, 1, 1, 1]
--------------------
[1, 1, 1, 0]